To dos:
- create a dataset with age and lon, lat and download it
- specify a new location to save images on the server
- loop over the dataset
 - plug in lat and lon into the api code below
 - extract all images for that location
 - name image according to age and its unique code after the last /

CNN
- rewatch simple CNN video how to extract age from name file
- run a simple CNN and compare results

Extent dataset:
- access Mapillary data either through API or the download you did
- repeat above excercise
- compare R2
- compare coverage of images between the two - are the duplicates?

In [12]:
import requests
import re
import json
import os

current_directory = r"c:\Users\kaius\Documents\Master Thesis\ufo-predict data\street view images"
# Define the API endpoint with the specific latitude and longitude
url = "https://api.openstreetcam.org/2.0/photo/?lat=52.51263046&lng=13.38953209"


# Send a GET request to the API
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Debug: Print the response JSON to find the correct key for the image URL
    #print(response.json())

    data = response.json()

    # Define a regular expression pattern to match the URLs
    url_pattern = r'https://storage\d+\.openstreetcam\.org/files/photo/\d+/\d+/\d+/[^"]+\.jpg'

    # Find all matches of the pattern
    urls = re.findall(url_pattern, json.dumps(data))
    #print(urls)

    # Filter URLs that include "{{sizeprefix}}", "proc", or "th" before the last "/" sign
    filtered_urls = []
    for url in urls:
    # Split the URL by "/" to get all segments
        parts = url.split('/')
    # Check if the segment before the last one matches any of the specified patterns
        if len(parts) > 1:  # Ensure there are enough parts to inspect
            segment_to_check = parts[-2]  # Get the segment before the last one
        # Check if the segment does not contain any of the specified patterns
            if not any(pattern in segment_to_check for pattern in ["{{sizeprefix}}", "proc"]) and (segment_to_check.endswith("th") and not segment_to_check.endswith("lth")):
                filtered_urls.append(url)
    
    print(filtered_urls)
    
    if filtered_urls:  
        # Check if image_url is not None
        for image_url in filtered_urls:
            print(image_url)
            file_name = image_url.split('/')[-1]
            if not file_name.endswith('.jpg'):
                file_name += '.jpg'
        # Combine the directory with the filename
            file_path = os.path.join(current_directory, file_name)
        # Send a GET request to fetch the image
            image_response = requests.get(image_url)
    
            if image_response.status_code == 200:
            # Save the image to a file
                with open(file_path, 'wb') as f:
                    f.write(image_response.content)
                print("Image downloaded successfully.")
            else:
                print("Failed to download the image.")
    else:
        print("Image URL was not found in the API response.")
else:
    print("Failed to retrieve data from the API.")



['https://storage2.openstreetcam.org/files/photo/2016/8/25/th/14682_72e79_57bf17a3d0fc7.jpg', 'https://storage6.openstreetcam.org/files/photo/2017/11/10/th/966533_dfcfa_13.jpg', 'https://storage6.openstreetcam.org/files/photo/2017/11/10/th/966541_6cb2b_25.jpg', 'https://storage13.openstreetcam.org/files/photo/2020/7/26/th/2818418_83ecb_5f1d8e994f5a5.jpg', 'https://storage13.openstreetcam.org/files/photo/2020/7/28/th/2827634_9008c_5f20513eb1ff1.jpg', 'https://storage13.openstreetcam.org/files/photo/2020/8/5/th/2865646_84ea2_5f2a2bff88b76.jpg', 'https://storage13.openstreetcam.org/files/photo/2020/8/5/th/2865742_9bb8d_5f2a3576d6d95.jpg', 'https://storage13.openstreetcam.org/files/photo/2020/8/15/th/2917218_41295_5f37ec1e71d2e.jpg', 'https://storage13.openstreetcam.org/files/photo/2020/8/23/th/2943810_b2100_5f41d7658e0a9.jpg', 'https://storage13.openstreetcam.org/files/photo/2020/10/28/th/3196014_31e7b_5f998cea88f8f.jpg', 'https://storage13.openstreetcam.org/files/photo/2020/10/29/th/3198

In [13]:
import requests
import os
# Get the current working directory
current_directory = r"c:\Users\kaius\Documents\Master Thesis\ufo-predict data\street view images"

# Specify the filename
filename = "downloaded_image.jpg"
# Combine the directory with the filename
file_path = os.path.join(current_directory, filename)

# Example image URL returned by the API
image_url = "https://storage6.openstreetcam.org/files/photo/2017/11/10/proc/966541_6cb2b_25.jpg"

# Send a GET request to fetch the image
image_response = requests.get(image_url)

if image_response.status_code == 200:
    # Save the image to a file
    with open(file_path, 'wb') as f:
        f.write(image_response.content)
    print("Image downloaded successfully.")
else:
    print("Failed to download the image.")


Image downloaded successfully.


In [ ]:
urls = "https://storage6.openstreetcam.org/files/photo/2017/11/10/lth/966541_6cb2b_25.jpg"
filtered_urls = []
for url in urls:
    parts = url.split('/')
    # Check if the segment before the last one matches any of the specified patterns
    if len(parts) > 1:  # Ensure there are enough parts to inspect
        segment_to_check = parts[-2]  # Get the segment before the last one
        # Check if the segment does not contain any of the specified patterns
        if not any(pattern in segment_to_check for pattern in ["{{sizeprefix}}", "proc", "th"]):
            filtered_urls.append(url)